# December 16, 2024

https://adventofcode.com/2024/day/16

In [9]:
from queue import PriorityQueue

In [56]:
DEBUG = False
def dprint( *args ):
    if DEBUG:
        return print(*args)


In [88]:
# direction dict
dirct = {
    "^": [-1,0],
    "v": [+1,0],
    "<": [0,-1],
    ">": [0,+1]
}

WALL = "#"
SPACE = "."
BOX = "O"
BOXL = "["
BOXR = "]"
ROBOT = "@"

In [31]:
test_str1 = f'''###############
#.......#....E#
#.#.###.#.###.#
#.....#.#...#.#
#.###.#####.#.#
#.#.#.......#.#
#.#.#####.###.#
#...........#.#
###.#.#####.#.#
#...#.....#.#.#
#.#.#.###.#.#.#
#.....#...#.#.#
#.###.#.#.#.#.#
#S..#.....#...#
###############'''
test_str1 = test_str1.split("\n")

test_str2 = f'''#################
#...#...#...#..E#
#.#.#.#.#.#.#.#.#
#.#.#.#...#...#.#
#.#.#.#.###.#.#.#
#...#.#.#.....#.#
#.#.#.#.#.#####.#
#.#...#.#.#.....#
#.#.#####.#.###.#
#.#.#.......#...#
#.#.###.#####.###
#.#.#...#.....#.#
#.#.#.#####.###.#
#.#.#.........#.#
#.#.#.#########.#
#S#.............#
#################'''
test_str2 = test_str2.split("\n")

In [32]:
fn = "../data/2024/16.txt"
with open(fn, "r") as file:
    text = file.readlines()
puzz_str = [x.strip() for x in text]

In [33]:
def parse_input( text ):
    walls = set()
    for r, line in enumerate(text):
        for c, x in enumerate(line):
            if x == "#":
                walls.add( (r,c) )
            elif x == "E":
                goal = (r,c)
            elif x == "S":
                start = (">", r, c)

    return {"start":start, "goal":goal, "walls":walls}

# Part 1


In [47]:
# direction dictionary
# gives dr, dc when moving in the given direction
dirct = {
    "^": [-1,0],
    "v": [+1,0],
    "<": [0,-1],
    ">": [0,+1]
}


In [48]:
def heuristic( goal, state ):
    '''
    Assumes no walls in the way: minimal number of moves and turns'''
    bearing, r, c = state
    gr, gc = goal

    moves = abs(gr-r) + abs(gc-c)

    turns = 0
    if gr < r:
        # need to go up: 1 turn from sideways, 2 turns from down
        turns += (bearing != "^") + (bearing == "v")
    if gr > r:
        # need to go down: 1 turn from sideways, 2 turns from up
        turns += (bearing != "v") + (bearing == "^")
    if gc < c:
        # need to go left: 1 turn from vertical, 2 turns from right
        turns += (bearing != "<") + (bearing == ">")
    if gc > c:
        # need to go right: 1 turn from vertical, 2 turns from left
        turns += (bearing != ">") + (bearing == "<")

    # The max number of turns you need is two, because if you're facing the opposite direction
    # You can do your orthogonal moves first.
    # for example, going upright while facing left... you can turn up, move up, turn right, move right
    turns = min(turns, 2)

    return 1000*turns + moves




In [49]:
def get_neighbors( state, walls ):
    '''
    Return the neighbor states
    This includes turning or moving, but not both
    '''
    bearing, r, c = state
    dr, dc = dirct[bearing]
    r2, c2 = (r + dr, c + dc)

    nbrs = set()
    if (r2, c2) not in walls:
        nbrs.add( (bearing, r2, c2) )

    if bearing in ("^", "v"):
        nbrs.add( ("<", r, c) )
        nbrs.add( (">", r, c) )
    else:
        nbrs.add( ("^", r, c) )
        nbrs.add( ("v", r, c) )

    return nbrs

In [50]:
def get_cost( current, next ):
    '''state = (bearing, row, col). This function assumes one move xor one turn'''    
    if current[0] == next[0]:
        cost = abs(current[1] - next[1]) + abs(current[2] - next[2])
    else:
        cost = 1000

    return cost

In [64]:
def Astar( start, goal, walls ):
   
   frontier = PriorityQueue()
   frontier.put( (0, start) )
   came_from = dict()
   cost_so_far = dict()
   came_from[start] = None
   cost_so_far[start] = 0


   while not frontier.empty():
      current = frontier.get()[1] #[0] is the priority, [1] is the value
      dprint(current)

      # ignore current[0] which is the bearing
      if current[1:] == goal:
         break

   
      for next in get_neighbors(current, walls):
         dprint(next)
         new_cost = cost_so_far[current] + get_cost(current, next)
      
         if next not in cost_so_far or new_cost < cost_so_far[next]:
            cost_so_far[next] = new_cost
            priority = new_cost + heuristic(goal, next)

            frontier.put( (priority, next) )
            came_from[next] = current

   return cost_so_far[current]

In [152]:
DEBUG = False
puzz = parse_input(test_str1)
Astar( **puzz )

7036

In [153]:
DEBUG = False
puzz = parse_input(test_str2)
Astar( **puzz )

11048

In [154]:
DEBUG = False
puzz = parse_input(puzz_str)
Astar( **puzz )

102460

# Part 2

In [175]:
def Astar_multi( start, goal, walls ):
   '''
   modified Astart to return all best paths
   '''
   frontier = PriorityQueue()
   frontier.put( (0, start) )
   came_from = dict()
   cost_so_far = dict()
   came_from[start] = None
   cost_so_far[start] = 0

   best_cost = None


   while not frontier.empty():
      item = frontier.get() #[0] is the priority, [1] is the value
      prio = item[0]
      current = item[1]
      
      dprint(prio, current)


      #* if prio > best_cost, then no more solutions exist
      if best_cost is not None and prio > best_cost:
         break

      #* don't quit early until we look for alternate solutions   
      if current[1:] == goal:
         best_cost = cost_so_far[current]
         continue

   
      for next in get_neighbors(current, walls):
         dprint(next)
         new_cost = cost_so_far[current] + get_cost(current, next)
      

         
         if next not in cost_so_far or new_cost < cost_so_far[next]:
               cost_so_far[next] = new_cost
               priority = new_cost + heuristic(goal, next)
               frontier.put( (priority, next) )
               came_from[next] = set((current,))
         elif new_cost == cost_so_far[next]:
               #* for ties, we add an alternate path, but don't update costs or priorities
               came_from[next].add((current))

   return came_from

In [176]:
def find_best_seats( came_from, state ):
    dprint(state)
    path = set( (state[1:],) )
    if came_from[state] is not None:

        for prev in came_from[state]:
            #dprint(prev)
            path = path.union( find_best_seats( came_from, prev ) )
    return path

def find_all_seats( came_from, goal ):
    goals = set( (("^", *goal), (">", *goal), ("v", *goal), ("<", *goal)) )
    goals = goals.intersection( set(came_from) )
    
    seats = set()
    for g in goals:
        seats = seats.union( find_best_seats(came_from, g) )
    seats.add(goal)

    return seats

In [181]:
def part2( text ):
    puzz = parse_input( text )
    came_from = Astar_multi( **puzz )
    seats = find_all_seats( came_from, puzz["goal"] )
    return len(seats)

In [182]:
part2(test_str1)

45

In [183]:
part2(test_str2)

64

In [185]:
part2(puzz_str)

527